In [15]:
import pandas as pd
import numpy as np
import datetime

In [187]:
# Question 1
o20 = pd.read_csv('./ArgusaChallenge2023/orders_2020.csv')
o21 = pd.read_csv('./ArgusaChallenge2023/orders_2021.csv')
o22 = pd.read_csv('./ArgusaChallenge2023/orders_2022.csv')
o23 = pd.read_csv('./ArgusaChallenge2023/orders_2023.csv')

products=pd.read_csv('./ArgusaChallenge2023/products.csv')
total_revenues = 46449091

In [53]:
  input_data=products
  sum = pd.DataFrame(input_data.dtypes, columns=['dtypes'])
  sum['missing#'] = input_data.isna().sum()
  sum['missing%'] = (input_data.isna().sum())/len(input_data)
  sum['uniques'] = input_data.nunique().values
  sum['count'] = input_data.count().values
print(sum)

              dtypes  missing#  missing%  uniques  count
category_1    object         0  0.000000        8   4323
category_2    object         0  0.000000       28   4323
category_3    object       455  0.105251       37   3868
category_4    object      2128  0.492251       31   2195
category_5    object      4124  0.953967        3    199
name          object         0  0.000000       90   4323
description   object       148  0.034235       77   4175
base_price   float64         0  0.000000       85   4323
start_date    object         0  0.000000     1286   4323
end_date      object        78  0.018043     1276   4245
sale_price   float64         0  0.000000     4083   4323
id             int64         0  0.000000       91   4323


In [201]:
p_startdate=pd.to_datetime(products['start_date']).values
p_startdate=np.array(p_startdate,dtype=np.datetime64)

p_enddate=pd.to_datetime(products['end_date']).values
p_enddate=np.array(p_enddate,dtype=np.datetime64)
total=0
for year in [2020]:
    if year==2020: data=pd.DataFrame.copy(o20)
    elif year==2021: data=pd.DataFrame.copy(o21)
    elif year==2022: data=pd.DataFrame.copy(o22)
    elif year==2023: data=pd.DataFrame.copy(o23)

    data['price_sold'] = None
    dates_year = pd.to_datetime(data['order_date']).values
    dates_year=np.array(dates_year,dtype=np.datetime64)

    prices = np.array([])
    for (ind_order, date) in enumerate(dates_year):
        ind_prod = data['product_id'][ind_order]
        ind_sold = np.where((date<p_enddate)&(date>=p_startdate)&(products['id'].values==ind_prod))
        if np.size(ind_sold)==0:
            prices=np.append(prices, 0)
            print(ind_order, date)
        else: 
            prices=np.append(prices, products['sale_price'][ind_sold[0]])
    data['price_sold']=prices
    total+=np.sum(prices)
    if year==2020: o20_new=data
    elif year==2021: o21_new=data
    elif year==2022: o22_new=data
    elif year==2023: o23_new=data

66 2020-06-11T00:00:00.000000000
129 2020-08-31T00:00:00.000000000
135 2020-09-17T00:00:00.000000000
162 2020-10-27T00:00:00.000000000
222 2020-05-10T00:00:00.000000000
238 2020-06-02T00:00:00.000000000
239 2020-06-02T00:00:00.000000000
250 2020-06-28T00:00:00.000000000
267 2020-08-11T00:00:00.000000000
297 2020-09-20T00:00:00.000000000
298 2020-09-20T00:00:00.000000000
299 2020-09-20T00:00:00.000000000
351 2020-11-22T00:00:00.000000000
352 2020-11-22T00:00:00.000000000
353 2020-11-22T00:00:00.000000000
389 2020-12-21T00:00:00.000000000
390 2020-12-21T00:00:00.000000000
437 2020-05-23T00:00:00.000000000
438 2020-05-23T00:00:00.000000000
466 2020-06-27T00:00:00.000000000
467 2020-06-27T00:00:00.000000000
468 2020-06-27T00:00:00.000000000
469 2020-06-27T00:00:00.000000000
479 2020-07-14T00:00:00.000000000
533 2020-10-12T00:00:00.000000000
552 2020-11-03T00:00:00.000000000
562 2020-11-16T00:00:00.000000000
563 2020-11-16T00:00:00.000000000
598 2020-02-08T00:00:00.000000000
644 2020-01-26T

7022 2020-04-17T00:00:00.000000000
7076 2020-07-16T00:00:00.000000000
7077 2020-07-16T00:00:00.000000000
7090 2020-07-28T00:00:00.000000000
7091 2020-07-28T00:00:00.000000000
7092 2020-07-28T00:00:00.000000000
7102 2020-08-16T00:00:00.000000000
7121 2020-09-21T00:00:00.000000000
7205 2020-02-19T00:00:00.000000000
7211 2020-03-05T00:00:00.000000000
7216 2020-03-24T00:00:00.000000000
7239 2020-04-30T00:00:00.000000000
7247 2020-05-20T00:00:00.000000000
7322 2020-09-21T00:00:00.000000000
7460 2020-03-12T00:00:00.000000000
7466 2020-04-26T00:00:00.000000000
7486 2020-05-25T00:00:00.000000000
7502 2020-06-19T00:00:00.000000000
7503 2020-06-19T00:00:00.000000000
7592 2020-11-11T00:00:00.000000000
7663 2020-02-13T00:00:00.000000000
7680 2020-04-04T00:00:00.000000000
7759 2020-08-06T00:00:00.000000000
7760 2020-08-06T00:00:00.000000000
7761 2020-08-06T00:00:00.000000000
7773 2020-08-24T00:00:00.000000000
7792 2020-10-31T00:00:00.000000000
7827 2020-11-30T00:00:00.000000000
7857 2020-12-23T00:0

13642 2020-11-27T00:00:00.000000000
13675 2020-12-23T00:00:00.000000000
13737 2020-04-24T00:00:00.000000000
13838 2020-10-27T00:00:00.000000000
13919 2020-12-27T00:00:00.000000000
13931 2020-02-07T00:00:00.000000000
13958 2020-04-29T00:00:00.000000000
14037 2020-08-25T00:00:00.000000000
14038 2020-08-25T00:00:00.000000000
14091 2020-11-03T00:00:00.000000000
14131 2020-12-13T00:00:00.000000000
14132 2020-12-13T00:00:00.000000000
14189 2020-01-13T00:00:00.000000000
14198 2020-02-16T00:00:00.000000000
14221 2020-04-05T00:00:00.000000000
14222 2020-04-05T00:00:00.000000000
14255 2020-06-19T00:00:00.000000000
14256 2020-06-19T00:00:00.000000000
14282 2020-09-30T00:00:00.000000000
14329 2020-12-02T00:00:00.000000000
14330 2020-12-02T00:00:00.000000000
14369 2020-01-28T00:00:00.000000000
14462 2020-09-01T00:00:00.000000000
14484 2020-10-06T00:00:00.000000000
14506 2020-11-03T00:00:00.000000000
14575 2020-12-29T00:00:00.000000000
14592 2020-02-21T00:00:00.000000000
14600 2020-03-19T00:00:00.00

20230 2020-02-17T00:00:00.000000000
20250 2020-05-21T00:00:00.000000000
20297 2020-08-29T00:00:00.000000000
20306 2020-09-12T00:00:00.000000000
20388 2020-12-05T00:00:00.000000000
20389 2020-12-05T00:00:00.000000000


In [199]:
print(total/total_revenues)

1.0955957024002903


In [202]:
for year in [2020]:
    if year==2020: data=pd.DataFrame.copy(o20)
    elif year==2021: data=pd.DataFrame.copy(o21)
    elif year==2022: data=pd.DataFrame.copy(o22)
    elif year==2023: data=pd.DataFrame.copy(o23)

    data['price_sold'] = None
    dates_year = pd.to_datetime(data['order_date']).values
    dates_year=np.array(dates_year,dtype=np.datetime64)
    print(dates_year)

['2020-01-19T00:00:00.000000000' '2020-01-19T00:00:00.000000000'
 '2020-01-21T00:00:00.000000000' ... '2020-12-04T00:00:00.000000000'
 '2020-12-05T00:00:00.000000000' '2020-12-05T00:00:00.000000000']


In [160]:
prizes = np.array([])
for (ind_order, date) in enumerate(dates_year[20380:20389]):
    ind_prod = data['product_id'][ind_order]
    ind_sold = np.where((date<=p_enddate)&(date>p_startdate)& (products['id'].values==ind_prod))
    if np.size(ind_sold)==0:
        prizes=np.append(prizes, 0)
    else:
        prizes=np.append(prizes, products['sale_price'][ind_sold[0]])
o20_new['price_sold']=prizes

27 2021-12-15T00:00:00.000000000
(array([1305]),)
27 2021-12-17T00:00:00.000000000
(array([1305]),)
27 2021-12-17T00:00:00.000000000
(array([1305]),)
27 2021-12-17T00:00:00.000000000
(array([1305]),)
27 2021-12-17T00:00:00.000000000
(array([1305]),)
27 2021-12-18T00:00:00.000000000
(array([1305]),)
27 2021-12-18T00:00:00.000000000
(array([1305]),)
27 2021-12-18T00:00:00.000000000
(array([1305]),)
27 2021-12-18T00:00:00.000000000
(array([1305]),)


ValueError: Length of values (9) does not match length of index (20390)